# 1. Imports, data and template prep

In [ ]:
!pip install pandas chromadb faiss-cpu sentence-transformers langchain langchain-openai
!pip install -qU langchain_community faiss-cpu

from typing import List
from langchain.vectorstores import FAISS, Chroma
from langchain.embeddings.base import Embeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

In [ ]:
import os
import pandas as pd
import requests

# Load CSVs
parts_df = pd.read_csv("./Data/parts.csv")
systems_df = pd.read_csv("./Data/systems.csv")
scenarios_df = pd.read_csv("./Data/automotive_scenarios.csv")

# Combine them into one list of documents
def make_text(row):
    return " | ".join(str(x) for x in row if pd.notnull(x))

texts = parts_df.apply(make_text, axis=1).tolist() + \
        systems_df.apply(make_text, axis=1).tolist() + \
        scenarios_df.apply(make_text, axis=1).tolist()

In [ ]:
class LocalServerEmbeddings(Embeddings):
    def __init__(self, base_url: str):
        self.base_url = base_url

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        response = requests.post(f"{self.base_url}/embeddings", json={"input": texts})
        return [item["embedding"] for item in response.json()["data"]]

    def embed_query(self, text: str) -> List[float]:
        response = requests.post(f"{self.base_url}/embeddings", json={"input": [text]})
        return response.json()["data"][0]["embedding"]

embedding = LocalServerEmbeddings(base_url="http://localhost:1234/v1")

In [ ]:
from langchain.docstore.document import Document

documents = [Document(page_content=t) for t in texts]

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = splitter.split_documents(documents)

In [ ]:
template = """Use the following context to answer the question.
If unsure, say "I don't know". Keep answers short. End with: "Thanks for asking!"
Context: {context}
Question: {question}
Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# 2. Testing phi-4 and deepseek r1 with FAISS/Chromadb  

In [ ]:
import os

use_faiss = True  # toggle this to switch between FAISS and Chroma

if use_faiss:
    vectorstore = FAISS.from_documents(splits, embedding)
else:
    persist_directory = 'chroma_store'
    if os.path.exists(persist_directory):
        import shutil
        shutil.rmtree(persist_directory)
    vectorstore = Chroma.from_documents(
        documents=splits,
        embedding=embedding,
        persist_directory=persist_directory
    )

In [ ]:
llm = ChatOpenAI(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio",
    model="phi-4"
)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

question = "What causes brake failure in vehicles?"
result = qa_chain({"query": question})

print("Answer:", result["result"])
print("\nSources:")
for doc in result["source_documents"]:
    print("-", doc.page_content[:150])

In [ ]:
llm_2 = ChatOpenAI(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio",
    model="deepseek-r1-distill-qwen-7b"
)

qa_chain_2 = RetrievalQA.from_chain_type(
    llm_2,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

question = "What causes brake failure in vehicles?"
result = qa_chain_2({"query": question})

print("Answer:", result["result"])
print("\nSources:")
for doc in result["source_documents"]:
    print("-", doc.page_content[:150])

# 3. Compare chromadb and FAISS using the same model

In [ ]:
# Compare chromadb and faiss answers using deepseek-r1-distill-qwen-7b
import os

# Chroma db
persist_directory = 'chroma_store'
        
if os.path.exists(persist_directory):
    import shutil
    shutil.rmtree(persist_directory)
    
chromadb_vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

# FAISS db
faiss_vectorstore = FAISS.from_documents(splits, embedding)

llm = ChatOpenAI(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio",
    model="deepseek-r1-distill-qwen-7b"
)

qa_chain_chroma = RetrievalQA.from_chain_type(
    llm,
    retriever=chromadb_vectorstore.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

qa_chain_faiss = RetrievalQA.from_chain_type(
    llm,
    retriever=faiss_vectorstore.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

question = "What causes brake failure in vehicles?"

result_chroma = qa_chain_chroma({"query": question})
result_faiss = qa_chain_faiss({"query": question})

# Print results for both vector stores
print("\n\Chroma Results:")
print("Answer:", result_chroma["result"])
print("\nSources:")
for doc in result_chroma["source_documents"]:
    print("-", doc.page_content[:150])

print("\n\nFAISS Results:")
print("Answer:", result_faiss["result"])
print("\nSources:")
for doc in result_faiss["source_documents"]:
    print("-", doc.page_content[:150])
    

# 4. Create and use a conversational agent

## Setup

In [ ]:
from typing import List
from langchain.vectorstores import Chroma
from langchain.embeddings.base import Embeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

import os
import pandas as pd
import requests

# Load CSVs
parts_df = pd.read_csv("./Data/parts.csv")
systems_df = pd.read_csv("./Data/systems.csv")
scenarios_df = pd.read_csv("./Data/automotive_scenarios.csv")

# Combine them into one list of documents
def make_text(row):
    return " | ".join(str(x) for x in row if pd.notnull(x))

texts = parts_df.apply(make_text, axis=1).tolist() + \
        systems_df.apply(make_text, axis=1).tolist() + \
        scenarios_df.apply(make_text, axis=1).tolist()
        
class LocalServerEmbeddings(Embeddings):
    def __init__(self, base_url: str):
        self.base_url = base_url

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        response = requests.post(f"{self.base_url}/embeddings", json={"input": texts})
        return [item["embedding"] for item in response.json()["data"]]

    def embed_query(self, text: str) -> List[float]:
        response = requests.post(f"{self.base_url}/embeddings", json={"input": [text]})
        return response.json()["data"][0]["embedding"]

embedding = LocalServerEmbeddings(base_url="http://localhost:1234/v1")

from langchain.docstore.document import Document

documents = [Document(page_content=t) for t in texts]

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = splitter.split_documents(documents)

template = """Use the following context to answer the question.
If unsure, say "I don't know". Keep answers short. End with: "Thanks for asking!"
Context: {context}
Question: {question}
Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# Chroma db
persist_directory = 'chroma_store'
    
chromadb_vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

llm = ChatOpenAI(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio",
    model="deepseek-r1-distill-qwen-7b"
)

qa_chain_chroma = RetrievalQA.from_chain_type(
    llm,
    retriever=chromadb_vectorstore.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

## Create the agent that will use the RAG tool and a chain to clarify

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from typing import List, Optional
from langchain_core.output_parsers import StrOutputParser
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate

@tool
def rag_qa(query: str) -> str:
    """
    Answer a user’s question by retrieving from the automotive knowledge base.
    
    Args:
        query: A natural‑language question about automotive parts, systems, or scenarios.
    Returns:
        A concise answer based on retrieved context.
    """
    result = qa_chain_chroma.invoke({"query": query})
    return result["result"]


clarify_template = ChatPromptTemplate.from_template(
    """I want to make sure I understand the user's question or request.
    
    Here is what they said: {query}

    First, determine if this message is ambiguous or needs clarification.
    If it's clear and specific enough to provide a good response, respond with just: "NO_CLARIFY"

    If it's ambiguous or missing important details, formulate ONE specific clarifying question that would help you provide a better response.

    Your response should be EITHER:
    1. "NO_CLARIFY" (if no clarification is needed)
    OR
    2. A single, concise clarifying question (if clarification is needed)
    """
)

clarify_chain = clarify_template | llm | StrOutputParser()

@tool
def clarify_llm(query: str) -> str:
    """
    Use an LLM to decide if the last user message needs clarification.
    Returns a follow-up question if needed, otherwise returns an empty string.
    """
    result = clarify_chain.invoke({"query": query})
    if "NO_CLARIFY" in result:
        return ""
    else:
        return result

tools = [
    rag_qa,
    clarify_llm
]

memory = MemorySaver()
config = {"configurable": {"thread_id": "abc123"}}

agent_executor = create_react_agent(
    llm, 
    tools, 
    checkpointer=memory)

response = agent_executor.invoke(
    {
        "messages": [HumanMessage(content="Hello")]
    }, 
    config
)

for message in response["messages"]:
    print(f"Type: {type(message).__name__}")
    print(f"Content: {message.content}")
    print("---")

Type: HumanMessage
Content: Hello
---
Type: AIMessage
Content: <think>
Okay, so I'm trying to figure out what to do when someone asks me a question that doesn't seem to match any of the available tools. Let's see... The tools provided are 'rag_qa' and 'clarify_llm'. 

First, I need to understand each tool. The 'rag_qa' function retrieves information from a knowledge base about automotive parts or scenarios. It takes a query as an argument. The 'clarify_llm' uses an LLM to decide if the user needs more clarification on their message.

Now, looking at the example given by the user: "Hello". That's just a greeting and doesn't ask for any specific information that these tools can provide. 

I should check if either tool is applicable here. The 'rag_qa' requires a query about automotive parts or systems, which isn't relevant to "Hello". Similarly, 'clarify_llm' is meant for determining if a message needs clarification, but in this case, the user just sent a greeting.

Since neither tool can